In [ ]:
#! pip install fairlearn
#! pip install lightgbm
#! pip install optuna

In [5]:
from sklearn.metrics import f1_score, confusion_matrix, make_scorer, accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split, cross_validate
from fairlearn.metrics import (
    count,
    selection_rate,
    MetricFrame,
    equalized_odds_difference,
    false_positive_rate,
    false_negative_rate,
    true_negative_rate,
    true_positive_rate,
    demographic_parity_difference
)

from fairlearn.datasets import fetch_adult
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

import numpy as np
import optuna
import tqdm as notebook_tqdm

In [6]:
data = fetch_adult(as_frame=True)
X_raw = data.data
y = (data.target == ">50K") * 1
A = X_raw["sex"]

numeric_transformer = Pipeline(
    steps=[
        ("impute", SimpleImputer()),
        ("scaler", StandardScaler()),
    ]
)
categorical_transformer = Pipeline(
    [
        ("impute", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(handle_unknown="ignore")),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, selector(dtype_exclude="category")),
        ("cat", categorical_transformer, selector(dtype_include="category")),
    ]
)

pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        (
            "classifier",
            LGBMClassifier(n_jobs=-1),
        ),
    ]
)

/home/azucena/miniconda3/envs/fairenv/lib/python3.10/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [7]:
metric_frame = MetricFrame(
    metrics=true_positive_rate, y_true=y_test, y_pred=y_pred, sensitive_features=A_test
)

NameError: name 'y_test' is not defined

In [8]:
def metric_scorer(clf, X, y):
    y_pred = clf.predict(X)
    f1 = f1_score(y,y_pred)
    abs_eod = np.abs(equalized_odds_difference(y, y_pred, sensitive_features=X['sex']))
    return {'f1_score': f1, 'eod': abs_eod}

def metric_scorer_v2(clf, X, y):
    y_pred = clf.predict(X)
    error = 1 - accuracy_score(y, y_pred)
    abs_dpd = np.abs(demographic_parity_difference(y, y_pred, sensitive_features=X['sex']))
    return {'error': error, 'dpd': abs_dpd}

def metric_scorer_v3(clf, X, y):
    y_pred = clf.predict(X)
    f1 = f1_score(y,y_pred)
    abs_dpd = np.abs(demographic_parity_difference(y, y_pred, sensitive_features=X['sex']))
    return {'f1_score': f1, 'dpd': abs_dpd}

def equality_opportunity_difference(y_true, y_pred, sensitive_features):
    return MetricFrame(metrics=true_positive_rate, y_true=y_true, y_pred=y_pred, sensitive_features=sensitive_features).difference()

def metric_scorer_v4(clf, X, y):
    y_pred = clf.predict(X)
    f1 = f1_score(y,y_pred)
    abs_tpr = np.abs(equality_opportunity_difference(y_true=y, y_pred=y_pred, sensitive_features=X['sex']))
    #abs_dpd = np.abs(demographic_parity_difference(y, y_pred, sensitive_features=X['sex']))
    return {'f1_score': f1, 'tpr': abs_tpr}


In [4]:
#from fairlearn.metrics import equalized_odds_difference
#import numpy as np

#pipeline.fit(X_train, y_train)
#y_pred = pipeline.predict(X_test)
#fair_value = np.abs(equalized_odds_difference(y_test, y_pred, sensitive_features=A_test))
#fair_value = np.abs(demographic_parity_difference(y_test, y_pred, sensitive_features=A_test))
#fair_value = np.abs(equality_opportunity_difference(y_test, y_pred, sensitive_features=A_test))
#model_value = f1_score(y_test, y_pred) 

#print(fair_value)
#print(model_value)

0.1908097330743628
0.6951638780655514


In [9]:
seed = 128
num_variables = 2
n_startup_trials = 11 * num_variables - 1
directions = ['minimize', 'maximize']
results = []

for sim in [0]:
    print(sim)
    def objective(trial):

        (X_train, X_test, y_train, y_test, A_train, A_test) = train_test_split(
        X_raw, y, A, test_size=0.8, random_state=sim, stratify=y
        )

        X_train = X_train.reset_index(drop=True)
        X_test = X_test.reset_index(drop=True)
        y_train = y_train.reset_index(drop=True)
        y_test = y_test.reset_index(drop=True)
        A_train = A_train.reset_index(drop=True)
        A_test = A_test.reset_index(drop=True)


        params = {
            'n_estimators': trial.suggest_int("n_estimators", 20, 10000),
            'num_leaves': trial.suggest_int("num_leaves", 10, 1000),
            'max_depth': trial.suggest_int("max_depth", 2, 20),
            'min_child_samples': trial.suggest_int("min_child_samples", 5, 300),
            'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-2),
            'boosting_type': trial.suggest_categorical("boosting_type", ['goss', 'gbdt'])
            }
        pipeline['classifier'].set_params(**params)

        scores = cross_validate(
                pipeline, 
                X_train,
                y_train, 
                cv=5,
                scoring = metric_scorer_v4,
                return_train_score=True)

        #fair_metric = scores['test_eod'].mean()
        #fair_metric = scores['test_dpd'].mean()
        fair_metric = scores['test_tpr'].mean()
        model_metric = scores['test_f1_score'].mean()

        return fair_metric, model_metric
    
    #sampler = optuna.samplers.MOTPESampler(
    #    n_startup_trials=n_startup_trials, n_ehvi_candidates=24, seed=seed
    #    )
    sampler = optuna.samplers.TPESampler()  # `MOTPESampler` used to be required for multi-objective optimization.
    study = optuna.create_study(directions=directions, pruner=optuna.pruners.SuccessiveHalvingPruner(), sampler = sampler)
    study.optimize(objective, n_trials=100)
    print("Number of finished trials: ", len(study.trials))
    results.append(study)
    #return fair_metric

[I 2023-05-10 13:58:07,497] A new study created in memory with name: no-name-583e4ab7-604b-4deb-8d11-9ecb5235a755


0


[I 2023-05-10 13:59:00,914] Trial 0 finished with values: [0.07166280837428934, 0.6700666103847778] and parameters: {'n_estimators': 6543, 'num_leaves': 578, 'max_depth': 12, 'min_child_samples': 203, 'learning_rate': 0.00899276083535639, 'boosting_type': 'gbdt'}. 
[I 2023-05-10 13:59:40,595] Trial 1 finished with values: [0.047887253714034086, 0.6748810216777641] and parameters: {'n_estimators': 3269, 'num_leaves': 733, 'max_depth': 14, 'min_child_samples': 254, 'learning_rate': 0.009178472770287455, 'boosting_type': 'gbdt'}. 
[I 2023-05-10 14:00:16,920] Trial 2 finished with values: [0.07566443079740351, 0.6681784105958057] and parameters: {'n_estimators': 1805, 'num_leaves': 648, 'max_depth': 19, 'min_child_samples': 52, 'learning_rate': 0.0031665827084322523, 'boosting_type': 'goss'}. 
[I 2023-05-10 14:01:02,417] Trial 3 finished with values: [0.059231349025974524, 0.6810983756692895] and parameters: {'n_estimators': 2189, 'num_leaves': 79, 'max_depth': 14, 'min_child_samples': 24,

In [35]:
import dill
file_name = 'f1-dpd-lgbm-motpe-succesivehalving-100trials-4sim.pkl'
#f1-eod-lgbm-succesivehalving-30trails.pkl
with open(file_name, 'wb') as file:
    dill.dump(results, file)
    print(f'Object successfully saved to "{file_name}"')

Object successfully saved to "f1-dpd-lgbm-motpe-succesivehalving-100trials-4sim.pkl"


In [29]:
optuna.visualization.plot_pareto_front(study, target_names=["fair_metric", "f1_score"], include_dominated_trials = True)